In [ ]:
from lxml import etree
from pyproj.crs import CRS
from pyproj.transformer import Transformer, TransformerGroup
import pandas as pd
import numpy as np

In [ ]:
# Crea el dataframe con los datos del relevamiento
relevamiento = './data/relevamiento.csv'
relevamiento_df = pd.read_csv(relevamiento, delimiter=",", header=0, encoding="utf-8")
print (relevamiento_df)

In [ ]:
# Crea las numpy array
relevamiento_arr = relevamiento_df.to_numpy()
faja5_etiquetas = relevamiento_arr[::,[0]]
faja5_puntos = relevamiento_arr[::,np.array([1,2,3])]
print('faja5_etiquetas = \n', faja5_etiquetas)
print('faja5_puntos = \n', faja5_puntos)

In [ ]:
# Define los sistemas de coordenadas
wgs84_crs = CRS.from_epsg(4979) # WGS84 (lat,lon,h)
faja5_crs = CRS.from_epsg(5347) # Posgar2007/ Faja 5 (N,E)
print('wgs84_crs = ', wgs84_crs)
print('faja5_crs = ', faja5_crs)

In [ ]:
# Usa la transformación EPSG:5351 (en vez de EPSG:9264)
faja5_to_wgs84 = TransformerGroup(faja5_crs, wgs84_crs).transformers[1]
print('faja5_to_wgs84 = \n', faja5_to_wgs84)

In [ ]:
# Transforma a WGS84 (la entrada es: N, E, Z)
wgs84_puntos = (np.array(faja5_to_wgs84.transform(
     faja5_puntos[::,0],
     faja5_puntos[::,1],
     faja5_puntos[::,2])).T)
print('wgs84_puntos = \n', wgs84_puntos)

In [ ]:
# Usa la grilla para convertir altitudes SVN16 a alturas elipsoidales WGS84
geoidar16_pipeline = r'+proj=pipeline +step +inv +proj=vgridshift +grids=./geoidear16/geoidear16.gtx'
geoidar16_transformer = Transformer.from_pipeline(geoidar16_pipeline)
print('geoidar16_transformer = \n', geoidar16_transformer)

In [ ]:
# Tranforma a alturas elipsoidales WGS84 (la entrada es lon,lat,h)
wgs84_corregidos = (np.array(geoidar16_transformer.transform(
    wgs84_puntos[::,1],
    wgs84_puntos[::,0],
    wgs84_puntos[::,2])).T)
print('wgs84_corregidos = \n', wgs84_corregidos)

In [ ]:
# Crea el dataframe de los puntos corregidos
wgs84_etiquetados = np.concatenate((faja5_etiquetas, wgs84_corregidos), axis=1)
wgs84_df = pd.DataFrame(wgs84_etiquetados,
                          columns=['marker_label',
                                   'wgs84_x',
                                   'wgs84_y',
                                   'wgs84_z'])
print('wgs84_df = \n', wgs84_df)

In [ ]:
# Lee el archivo de PAFs y crea el objeto 'chunk'
with open('./data/pafs.xml') as pafs:
    pafs_content = pafs.read().encode()
    pafs_xml = etree.XML(pafs_content)[0]
    print(pafs_xml)

In [ ]:
# Crea el dataframe con la rama 'cameras'
cameras_element = pafs_xml.find('cameras')
cameras_n = len(cameras_element)
cameras_attrs = ['id', 'label']
cameras_list = [[cameras_element[camera].get(attr)
                 for attr in cameras_attrs]
                    for camera in range(cameras_n)]
cameras_df = pd.DataFrame(cameras_list,
                          columns=['camera_id',
                                   'camera_label'])
print('cameras_df = \n', cameras_df)

In [ ]:
# Crea el dataframe con la rama 'markers'
markers_element = pafs_xml.find('markers')
markers_purged = markers_element.findall(".//reference[@enabled='true']")
markers_n = len(markers_purged)
markers_attrs = ['id', 'label']
references_attrs = ['x', 'y', 'z', 'enabled']
markers_list = [
    [markers_purged[marker].getparent().get(attr)
                for attr in markers_attrs] + 
    [markers_purged[marker].get(attr)
                 for attr in references_attrs]
                    for marker in range(markers_n)]
markers_df = pd.DataFrame(markers_list,
                          columns=['marker_id',
                                   'marker_label',
                                   'marker_x',
                                   'marker_y',
                                   'marker_z',
                                   'marker_enabled'],)
print('markers_df = \n', markers_df)
print(markers_df[['marker_label','marker_x','marker_y','marker_z']])

In [ ]:
# Combina las coordenadas WGS84 calculadas en las 'markers'
markers_wgs84_df = pd.merge(
    markers_df,
    wgs84_df,
    how = 'inner',
    on = 'marker_label')[['marker_id',
                          'marker_label',
                          'wgs84_x',
                          'wgs84_y',
                          'wgs84_z']]
print('markers_wgs84_df = \n', markers_wgs84_df)

In [ ]:
# Crea el dataframe con cada elemento ('location') de la rama 'frames'
frame0_element = pafs_xml.find('frames')[0]
frame0_markers_locations = frame0_element.findall(".//marker//location[@pinned='true']")
locations_attrs = ['camera_id', 'pinned', 'x', 'y']
locations_n = len(frame0_markers_locations)
locations_list =[
    [frame0_markers_locations[location].getparent().get('marker_id')]+
    [frame0_markers_locations[location].get(attr)
        for attr in locations_attrs]
            for location in range(locations_n)]
locations_df = pd.DataFrame(locations_list,
                            columns=['marker_id',
                                     'camera_id',
                                     'location_pinned',
                                     'camera_x',
                                     'camera_y'])
print('locations_df = \n', locations_df)

In [ ]:
# Combina las coordenadas de las 'markers' y las de las 'cameras' en cada 'location'
combinado = pd.merge(
    pd.merge(
        locations_df,
        cameras_df,
        how ='inner',
        on ='camera_id'),
    markers_wgs84_df,
    how ='inner',
    on = 'marker_id')[['wgs84_x',
                       'wgs84_y',
                       'wgs84_z',
                       'camera_x',
                       'camera_y',
                       'camera_label',
                       'marker_label'
                       ]]
print('combinado = \n', combinado)

In [ ]:
# Cambia a tipo float las columnas de coordenadas
float_combinado = combinado.astype({'wgs84_x': float,
                                    'wgs84_y': float,
                                    'wgs84_z': float})
print(float_combinado.dtypes)

In [ ]:
# Redondea
rounded_combinado = float_combinado.round({'wgs84_x': 8,'wgs84_y': 8,'wgs84_z': 3})
print('rounded_combinado = \n', rounded_combinado)

In [ ]:
# Exporta
with open('./output/gcp_list.txt', 'w') as archivo_salida:
    archivo_salida.write('EPSG:4326\n')
    rounded_combinado.to_csv(archivo_salida,
                             mode='a',
                             sep=' ',
                             header=False,
                             index=False)